In [86]:
from bs4 import BeautifulSoup as bs
import requests
import os
import re
import subprocess
from dotenv import load_dotenv
import customtkinter as ctk


login_url = "https://anitaku.pe/login.html"
base_url = """https://anitaku.pe/category/saenai-heroine-no-sodatekata-fine"""
serie = """SaekanoFine"""

load_dotenv('.secrets')

email = os.getenv('EMAIL')
password = os.getenv('PASSWORD')
file_path = os.getenv('FILE_PATH')

quality = 4
lastep = 5
idm = True # Sin idm, 5 episodios de máxima calidad tardan aproximadamente 15 minutos, con IDM tardan aproximadamente 3 minutos
epiString = True

In [87]:
print("base_url:", base_url)
print("serie:", serie)
print("quality:", quality)
print("lastep:", lastep)
print("idm:", idm)

base_url: https://anitaku.pe/category/saenai-heroine-no-sodatekata-fine
serie: SaekanoFine
quality: 4
lastep: 5
idm: True


In [88]:
episodios_descargas = {}

def transformar_url(url_base, numero_episodio):
    url_modificada = re.sub(r"/category", "", url_base)
    if(epiString):
        episodio_url = f"{url_modificada}-episode-{numero_episodio}"
    else:
        episodio_url = f"{url_modificada}-{numero_episodio}"
    print(episodio_url)
    return episodio_url

session = requests.Session()
login_page = session.get(login_url)
soup = bs(login_page.text, 'html.parser')
csrf_token = soup.find('input', {'name': '_csrf'})['value']

login_data = {
    "email": email,
    "password": password,
    "_csrf": csrf_token,
    "remember": "1"
}

login_response = session.post(login_url, data=login_data)


In [89]:
if (epiString):    
    if (idm):
        if "Logout" in login_response.text:
            print("Login exitoso!")
            
            def obtener_links_descarga(session, url):
                response = session.get(url)
                
                if response.status_code == 200:
                    soup = bs(response.content, 'html.parser')
                    list_download_div = soup.find('div', class_='list_dowload')
                    
                    if list_download_div:
                        cf_download_div = list_download_div.find('div', class_='cf-download')
                        
                        if cf_download_div:
                            links = [a['href'] for a in cf_download_div.find_all('a', href=True)]
                            if 1 <= quality <= len(links):
                                return links[quality - 1]
                            else:
                                print("La calidad seleccionada está fuera de rango.")
                                return links[len(links)-1]
                        else:
                            print("No se encontró el div 'cf-download'.")
                    else:
                        print("No se encontró el div 'list_dowload'.")
                else:
                    print(f"Error al acceder a la página. Código de estado: {response.status_code}")
                
                return None

            for i in range(0, lastep):
                episodio_url = transformar_url(base_url, i)
                link_descarga = obtener_links_descarga(session, episodio_url)
                
                if link_descarga:
                    episodios_descargas[episodio_url] = link_descarga
                    print(f"Episodio {i}: {link_descarga}")
                else:
                    episodios_descargas[episodio_url] = link_descarga
                    print(f"No se encontró un link de descarga para el episodio {i}")
            
            print("\nDiccionario de episodios con sus links de descarga:")
            print(episodios_descargas)
            
            def descargar_con_idm(episodios_dict):
                idm_path = r"C:\Program Files (x86)\Internet Download Manager\IDMan.exe"  # Ruta de IDM, ajusta si es necesario
                if not os.path.exists(idm_path):
                    print("IDM no se encontró en la ruta especificada.")
                    return
                
                for episodio, url_descarga in episodios_dict.items():
                    if url_descarga:
                        numero_episodio = re.search(r'-episode-(\d+)(?!.*\d)', episodio).group(1)
                        file_name = f"{serie}_Episode_{numero_episodio}.mp4"
                        file_full_path = os.path.join(file_path, file_name)
                        
                        print(f"Descargando {file_name} desde {url_descarga} ...")
                        
                        subprocess.run([idm_path, "/d", url_descarga, "/p", file_path, "/f", file_name, "/n", "/a"])
                        
                # Comienza todas las descargas agregadas a la cola
                subprocess.run([idm_path, "/s"])
                print("Todas las descargas se han añadido a IDM.")

            descargar_con_idm(episodios_descargas)

        else:
            print("Error en el login")

    else:
        if "Logout" in login_response.text:
            print("Login exitoso!")
            
            def obtener_links_descarga(session, url):
                response = session.get(url)
                
                if response.status_code == 200:
                    soup = bs(response.content, 'html.parser')
                    list_download_div = soup.find('div', class_='list_dowload')
                    
                    if list_download_div:
                        cf_download_div = list_download_div.find('div', class_='cf-download')
                        
                        if cf_download_div:
                            links = [a['href'] for a in cf_download_div.find_all('a', href=True)]
                            if 1 <= quality <= len(links):
                                return links[quality - 1]
                            else:
                                print("La calidad seleccionada está fuera de rango.")
                                return links[len(links)-1]
                        else:
                            print("No se encontró el div 'cf-download'.")
                    else:
                        print("No se encontró el div 'list_dowload'.")
                else:
                    print(f"Error al acceder a la página. Código de estado: {response.status_code}")
                
                return None

            for i in range(0, lastep):
                episodio_url = transformar_url(base_url, i)
                link_descarga = obtener_links_descarga(session, episodio_url)
                
                if link_descarga:
                    episodios_descargas[episodio_url] = link_descarga
                    print(f"Episodio {i}: {link_descarga}")
                else:
                    episodios_descargas[episodio_url] = link_descarga
                    print(f"No se encontró un link de descarga para el episodio {i}")
            
            print("\nDiccionario de episodios con sus links de descarga:")
            print(episodios_descargas)
            
            def descargar_episodios(episodios_dict):
                if not os.path.exists(file_path):
                    os.makedirs(file_path)
                
                for episodio, url_descarga in episodios_dict.items():
                    if url_descarga:
                        numero_episodio = re.search(r'-episode-(\d+)(?!.*\d)', episodio).group(1)
                        file_name = f"{serie}_Episode_{numero_episodio}.mp4"
                        file_full_path = os.path.join(file_path, file_name)
                        
                        print(f"Descargando {file_name} desde {url_descarga} ...")
                        
                        with session.get(url_descarga, stream=True) as response:
                            response.raise_for_status()
                            with open(file_full_path, 'wb') as file:
                                for chunk in response.iter_content(chunk_size=8192):
                                    file.write(chunk)
                        
                        print(f"Descarga de {file_name} completada.")
            
            descargar_episodios(episodios_descargas)

        else:
            print("Error en el login")
else:
    if (idm):
        if "Logout" in login_response.text:
            print("Login exitoso!")
            
            def obtener_links_descarga(session, url):
                response = session.get(url)
                
                if response.status_code == 200:
                    soup = bs(response.content, 'html.parser')
                    list_download_div = soup.find('div', class_='list_dowload')
                    
                    if list_download_div:
                        cf_download_div = list_download_div.find('div', class_='cf-download')
                        
                        if cf_download_div:
                            links = [a['href'] for a in cf_download_div.find_all('a', href=True)]
                            if 1 <= quality <= len(links):
                                return links[quality - 1]
                            else:
                                print("La calidad seleccionada está fuera de rango.")
                                return links[len(links)-1]
                        else:
                            print("No se encontró el div 'cf-download'.")
                    else:
                        print("No se encontró el div 'list_dowload'.")
                else:
                    print(f"Error al acceder a la página. Código de estado: {response.status_code}")
                
                return None

            for i in range(0, lastep):
                episodio_url = transformar_url(base_url, i)
                link_descarga = obtener_links_descarga(session, episodio_url)
                
                if link_descarga:
                    episodios_descargas[episodio_url] = link_descarga
                    print(f"Episodio {i}: {link_descarga}")
                else:
                    episodios_descargas[episodio_url] = None
                    print(f"No se encontró un link de descarga para el episodio {i}")
            
            print("\nDiccionario de episodios con sus links de descarga:")
            print(episodios_descargas)
            
            def descargar_con_idm(episodios_dict):
                idm_path = r"C:\Program Files (x86)\Internet Download Manager\IDMan.exe"  # Ruta de IDM, ajusta si es necesario
                if not os.path.exists(idm_path):
                    print("IDM no se encontró en la ruta especificada.")
                    return
                
                for episodio, url_descarga in episodios_dict.items():
                    if url_descarga:
                        numero_episodio = re.search(r'(\d+)(?!.*\d)', episodio).group(1)
                        file_name = f"{serie}_Episode_{numero_episodio}.mp4"
                        file_full_path = os.path.join(file_path, file_name)
                        
                        print(f"Descargando {file_name} desde {url_descarga} ...")
                        
                        subprocess.run([idm_path, "/d", url_descarga, "/p", file_path, "/f", file_name, "/n", "/a"])

                # Comienza todas las descargas agregadas a la cola
                subprocess.run([idm_path, "/s"])
                print("Todas las descargas se han añadido a IDM.")

            descargar_con_idm(episodios_descargas)

        else:
            print("Error en el login")

    else:
        if "Logout" in login_response.text:
            print("Login exitoso!")
            
            def obtener_links_descarga(session, url):
                response = session.get(url)
                
                if response.status_code == 200:
                    soup = bs(response.content, 'html.parser')
                    list_download_div = soup.find('div', class_='list_dowload')
                    
                    if list_download_div:
                        cf_download_div = list_download_div.find('div', class_='cf-download')
                        
                        if cf_download_div:
                            links = [a['href'] for a in cf_download_div.find_all('a', href=True)]
                            if 1 <= quality <= len(links):
                                return links[quality - 1]
                            else:
                                print("La calidad seleccionada está fuera de rango.")
                                return links[len(links)-1]
                        else:
                            print("No se encontró el div 'cf-download'.")
                    else:
                        print("No se encontró el div 'list_dowload'.")
                else:
                    print(f"Error al acceder a la página. Código de estado: {response.status_code}")
                
                return None

            for i in range(0, lastep):
                episodio_url = transformar_url(base_url, i)
                link_descarga = obtener_links_descarga(session, episodio_url)
                
                if link_descarga:
                    episodios_descargas[episodio_url] = link_descarga
                    print(f"Episodio {i}: {link_descarga}")
                else:
                    episodios_descargas[episodio_url] = None
                    print(f"No se encontró un link de descarga para el episodio {i}")
            
            print("\nDiccionario de episodios con sus links de descarga:")
            print(episodios_descargas)
            
            def descargar_episodios(episodios_dict):
                if not os.path.exists(file_path):
                    os.makedirs(file_path)
                
                for episodio, url_descarga in episodios_dict.items():
                    if url_descarga:
                        numero_episodio = re.search(r'(\d+)(?!.*\d)', episodio).group(1)
                        file_name = f"{serie}_Episode_{numero_episodio}.mp4"
                        file_full_path = os.path.join(file_path, file_name)
                        
                        print(f"Descargando {file_name} desde {url_descarga} ...")
                        
                        with session.get(url_descarga, stream=True) as response:
                            response.raise_for_status()
                            with open(file_full_path, 'wb') as file:
                                for chunk in response.iter_content(chunk_size=8192):
                                    file.write(chunk)
                        
                        print(f"Descarga de {file_name} completada.")
            
            descargar_episodios(episodios_descargas)

        else:
            print("Error en el login")

Login exitoso!
https://anitaku.pe/saenai-heroine-no-sodatekata-fine-episode-0
Error al acceder a la página. Código de estado: 404
No se encontró un link de descarga para el episodio 0
https://anitaku.pe/saenai-heroine-no-sodatekata-fine-episode-1
Episodio 1: https://ggredi.info/download.php?url=aHR0cHM6LyAawehyfcghysfdsDGDYdgdsfsdfwstdgdsgtert9URASDGHUSRFSJGYfdsffsderFStewthsfSFtrftesdfmOHp6YmhuN3B1LmFuZjU5OC5jb20vdXNlcjEzNDIvNThhMDU3NjUyZjhhNzgwMDQ0NzZjZDAzZGU1NDY5OWMvRVAuMS52MC4xNjM5NDU0NTk3LjEwODBwLm1wND90b2tlbj14SUhKaFpEdG9jaWV0M19iWDZxQ3pBJmV4cGlyZXM9MTcyNzg0MjA4NiZpZD0xNDYxNzAmdGl0bGU9KDE5MjB4MTA4MC1nb2dvYW5pbWUpc2FlbmFpLWhlcm9pbmUtbm8tc29kYXRla2F0YS1maW5lLWVwaXNvZGUtMS5tcDQ=
https://anitaku.pe/saenai-heroine-no-sodatekata-fine-episode-2
Error al acceder a la página. Código de estado: 404
No se encontró un link de descarga para el episodio 2
https://anitaku.pe/saenai-heroine-no-sodatekata-fine-episode-3
Error al acceder a la página. Código de estado: 404
No se encontró un link de